In [73]:
import numpy as np
import pandas as pd
import torch
import zipfile
import kaggle
import os
import math

from pathlib import Path

In [178]:
print('hello we building random forest algo')

hello we building random forest algo


In [179]:
# Download dataset to use locally

path = Path('titanic')
extract_path = Path('data/titanic')
if not path.exists():
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(extract_path)
    
assert os.path.isdir(extract_path) == True, f"Directory '{path}' should be created at this point"
assert len(os.listdir(extract_path)) > 0, f"Directory '{path}' is empty somehow, we expect files here"

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [180]:
df = pd.read_csv(f'{extract_path}/train.csv')
# df.head()

In [218]:
# Handle missing values in the dataset

modes = df.mode().iloc[0]
df.fillna(modes, inplace=True)
# df.isna().sum()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,B96 B98,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,B96 B98,S


In [182]:
# features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
# For now we will work just with few cols, later we could try to extend
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
# sibsp	-- # of siblings / spouses aboard the Titanic
# parch -- # of parents / children aboard the Titanic
# embarked -- Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

In [183]:
df['Sex'] = df['Sex'].map({'male': 1, 'female': 0}).astype('Int8')

In [184]:
train_index = int(np.floor(0.9 * len(df)))

X_tr = df[features][:train_index]
Y_tr = df['Survived'][:train_index].values
X_test = df[features][train_index:]
Y_test = df['Survived'][train_index:].values

In [185]:
X_tr.shape

(801, 6)

In [186]:
i_to_column = {i: c for i,c in enumerate(X_tr.columns)}
i_to_column

{0: 'Pclass', 1: 'Sex', 2: 'Age', 3: 'SibSp', 4: 'Parch', 5: 'Fare'}

In [187]:
#convert df to tenrsors
if isinstance(X_tr, pd.DataFrame):
    X_tr = torch.tensor(X_tr.astype(np.float32).values, dtype=torch.float32)
    Y_tr = torch.tensor(Y_tr, dtype=torch.float32)

In [188]:
def oob_score(tree, x, y):
    mis_label = 0
    for i in range(len(x)):
        pred = predict_tree(tree, x[i])
        if pred != y[i]:
            mis_label += 1
    return mis_label / len(x)

In [189]:
def entropy(p):
    '''
    How uncertain our split is. If values are 50/50 -- very uncertain, 1. 
    If nearly all values are the same (99/1) -- certainty is high, very low entropy value
    '''
    if p == 0 or p == 1:
        return 0
        
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

assert entropy(.5) == 1
assert entropy(.99) < 0.1

In [190]:
def information_gain(left_child, right_child):
    '''
    Calculates how much uncertainty is reduced by doing the split. Higher gain = better split.
    Assumes binary classification with labels 0 and 1.
    '''

    # Combine both child sets to get the parent
    parent = torch.cat((left_child, right_child), dim=0)

    def proportion_ones(t):
        if t.numel() == 0:
            return 0
        return torch.sum(t == 1).item() / t.numel() 

    # Proportions of class 1
    p_parent = proportion_ones(parent)
    p_left = proportion_ones(left_child)
    p_right = proportion_ones(right_child)

    # Entropies
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)

    # Weights
    l_weight = left_child.numel() / parent.numel()
    r_weight = right_child.numel() / parent.numel()

    return IG_p - l_weight * IG_l - r_weight * IG_r

In [216]:
class DecisionTree:
    def __init__(self, x, y,max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

        # ===== subset of features for each tree =====
        fts_len = len(features)
        fts_indices = range(0, fts_len)
        n_fts = round(fts_len * .75)
        self.fts = np.random.choice(fts_indices, n_fts, replace=False)


        # ===== bagging =====
        # TODO: potential easier way to init this stuff but i'm not sure rn
        # n = len(x)
        # indices = np.random.choice(n, size=n, replace=True)
        # # out-of-bag, for validation
        # oob_indices = [i for i in range(len(x)) if i not in indices]
        # return x[indices], y[indices], x[oob_indices], y[oob_indices]

        x_len = len(x)
        bootstrap_indices = list(set((np.random.choice(range(x_len), x_len, replace=True))))
        # out-of-bag, for validation
        oob_indices = [i for i in range(len(x)) if i not in bootstrap_indices]
        self.X = x[bootstrap_indices]
        self.Y = y[bootstrap_indices]
        self.X_oob = x[oob_indices]
        self.y_oob = y[oob_indices]

    def the_loop(self):
        for ft_index in self.fts:
            self.find_best_split(ft_index)

        

    def find_best_split(self, feature_index, depth=0):
        x = self.X
        y = self.Y
        
        if depth == self.max_depth:
            return 0

        best_split = None
        best_feature_index = None
        ig_max = -9999
        
        possible_splits = x[:, feature_index].unique()

        for split in possible_splits:
            left_mask = x[:, feature_index] <= split
            right_mask = x[:, feature_index] > split
            
            left = x[left_mask]
            right = x[right_mask]
            
            ig = information_gain(left, right)
            
            if ig > ig_max:
                ig_max = ig
                best_split = split
                best_feature_index = feature_index

        print(best_split)
        return best_split

In [217]:
tree = DecisionTree(x=X_tr, y=Y_tr)
tree.the_loop()

tensor(33.)
tensor(1.)
tensor(14.)
tensor(0.)
